In [1]:
import os
import re
import csv
from bs4 import BeautifulSoup

generated_guis_path = '../generated_guis'
app_ids = ['12740', '14283', '18782', '20947', '22151', '27360', '27382', '27707', '30982', '31390', '32310', '3261', '33383', '34346', '34517', '34527', '35526', '3727', '37505', '38961', '40673', '43872', '43977', '44756', '47926', '49794', '53054', '53469', '54377', '54468', '56905', '58124', '59429', '59576', '61851', '63575', '64858', '65592', '67044', '68368', '69574', '69587', '70410', '8640']
method_names = ['instruction', 'pd_zs', 'pd_fs', 'ref_instruction']
dimension_target = (375, 667)

In [2]:
def get_html_files():
    files = []
    for app_id in app_ids:
        for method in method_names:
            html_path = os.path.join(generated_guis_path, str(app_id), f'{method}.html')
            if os.path.isfile(html_path):
                files.append({'UI_Number': app_id, 'Method': method, 'path': html_path})
    return files

def analyze_html(fileinfo):
    with open(fileinfo['path'], encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    # All buttons (button, input[type=button|submit|reset])
    buttons = soup.find_all('button')
    input_buttons = soup.find_all('input', {'type': re.compile('button|submit|reset', re.I)})
    all_buttons = buttons + input_buttons
    diff_buttons = set()

    for b in all_buttons:
        key = (b.get('id'), b.get('name'), b.get_text(strip=True), b.get('value'))
        diff_buttons.add(key)

    # Clickable buttons (with onclick or type submit/reset/button)
    clickable_buttons = [b for b in all_buttons if b.has_attr('onclick') or b.get('type') in ['submit', 'button', 'reset']]

    # Action buttons (with event: onclick, onsubmit, or inside <form>)
    action_buttons = []
    for b in all_buttons:
        if b.has_attr('onclick') or b.has_attr('onsubmit'):
            action_buttons.append(b)
        # Button inside form (submit action)
        parent = b.find_parent('form')
        if parent:
            action_buttons.append(b)

    # All input fields
    input_fields = soup.find_all('input')
    textareas = soup.find_all('textarea')
    selects = soup.find_all('select')
    all_inputs = input_fields + textareas + selects
    diff_inputs = set()
    for inp in all_inputs:
        key = (inp.get('id'), inp.get('name'), inp.get('type'))
        diff_inputs.add(key)

    # Dimension – check for exact dimension in body, html, main container, or iframe
    dimension_found = False
    for tag in [soup.body, soup.html]:
        if tag:
            w = tag.get('width') or tag.get('style')
            h = tag.get('height') or tag.get('style')
            # Typically style="width:375px; height:667px;"
            if w and h:
                if ('375' in str(w) and '667' in str(h)) or ('375' in str(h) and '667' in str(w)):
                    dimension_found = True
    # Occasionally set in div with id/root/container
    for div in soup.find_all('div'):
        style = div.get('style')
        if style and '375px' in style and '667px' in style:
            dimension_found = True

    return {
        'UI_Number': fileinfo['UI_Number'],
        'Method': fileinfo['Method'],
        'All_inputs': len(all_inputs),
        'Diff_inputs': len(diff_inputs),
        'All_buttons': len(all_buttons),
        'Diff_buttons': len(diff_buttons),
        'Clickable_buttons': len(clickable_buttons),
        'Action_buttons': len(set(action_buttons)),
        'Dimension_375x667': 'YES' if dimension_found else 'NO'
    }

In [3]:
html_files = get_html_files()
results = []
for f in html_files:
    res = analyze_html(f)
    results.append(res)

with open('gui_prototype_analysis.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['UI_Number', 'Method', 'All_inputs', 'Diff_inputs',
                  'All_buttons', 'Diff_buttons', 'Clickable_buttons', 'Action_buttons', 'Dimension_375x667']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    for r in results:
        writer.writerow(r)

print("Analysis complete. Results saved to gui_prototype_analysis.csv")

Analysis complete. Results saved to gui_prototype_analysis.csv
